<a href="https://colab.research.google.com/github/twoheartKES/weather-sales-forecast-ai/blob/main/weather_sales_forecast_ai_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. /2023 폴더에서 CSV 파일들 읽기

2. 파일명에서 지역명, 지역코드 파싱해서 컬럼으로 추가

3. combined_df로 전체 통합

4. map_region_to_base()로 기준지역(동두천/파주/수원/양평/이천/서울) 매핑

5. 기준지역 + SUBTITLE 기준으로 COLUMN1~53 합계

6. 기준지역 → 지역명/지역코드로 바꿔서 final_df 만들고 CSV 저장

In [ ]:
print("all_files:", all_files)

all_files: ['kdca_2023_week_서울.csv', 'kdca_2023_week_경기_안양시_동안구_101.csv', 'kdca_2023_week_경기_의왕시_111.csv', 'kdca_2023_week_경기_수원시_권선구_093.csv', 'kdca_2023_week_경기_부천시_130.csv', 'kdca_2023_week_경기_고양시_덕양구_076.csv', 'kdca_2023_week_경기_포천시_116.csv', 'kdca_2023_week_경기_안성시_100.csv', 'kdca_2023_week_경기_고양시_일산동구_077.csv', 'kdca_2023_week_경기_의정부시_112.csv', 'kdca_2023_week_경기_평택시_115.csv', 'kdca_2023_week_경기_구리시_082.csv', 'kdca_2023_week_경기_양주시_103.csv', 'kdca_2023_week_경기_시흥시_097.csv', 'kdca_2023_week_경기_부천시_오정구_088.csv', 'kdca_2023_week_경기_하남시_117.csv', 'kdca_2023_week_경기_김포시_084.csv', 'kdca_2023_week_경기_이천시_113.csv', 'kdca_2023_week_경기_동두천시_086.csv', 'kdca_2023_week_경기_여주시_105.csv', 'kdca_2023_week_경기_성남시_수정구_091.csv', 'kdca_2023_week_경기_안산시_상록구_099.csv', 'kdca_2023_week_경기_고양시_일산서구_078.csv

In [ ]:
def parse_region_from_filename(filename: str):
    """
    예)
    - kdca_2023_week_경기_안양시_동안구_101.csv
    - kdca_2023_week_서울.csv
    두 패턴 모두 처리해서 (지역명, 지역코드)를 리턴.
    """
    base = filename.replace('.csv', '')
    parts = base.split('_')

    # 안전장치: 최소 길이 확인
    if len(parts) < 4:
        return None, None  # 예상 밖 패턴이면 둘 다 None

    # 서울처럼 코드 없이 지역만 있는 경우: kdca_2023_week_서울
    if len(parts) == 4:
        region_name = parts[3]   # '서울'
        region_code = None       # 코드 없음 (나중에 기준지역 매핑에서 세팅)
    else:
        # 경기 세부 지역처럼 코드를 포함하는 일반 패턴
        region_code = parts[-1]
        region_name_parts = parts[3:-1]
        region_name = '_'.join(region_name_parts)

    return region_name, region_code


In [ ]:
import os
import pandas as pd

csv_directory = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023'

# 1) 디렉토리 안의 CSV 파일 목록 가져오기
all_files = os.listdir(csv_directory)
csv_files = [os.path.join(csv_directory, f)
             for f in all_files if f.lower().endswith('.csv')]

print("Found CSV files:")
for f in csv_files:
    print(f)

# 2) 기대하는 컬럼 목록 정의
column_numbers = [f'COLUMN{i}' for i in range(1, 54)]  # COLUMN1 ~ COLUMN53
expected_columns = ['지역명', '지역코드'] + column_numbers + ['TITLE', 'SUBTITLE']

all_dataframes = []

for file_path in csv_files:
    filename = os.path.basename(file_path)

    region_name, region_code = parse_region_from_filename(filename)

    # CSV 읽기 (인코딩 에러나면 encoding='cp949' 등으로 바꾸기)
    df = pd.read_csv(file_path)

    # 앞에 지역명, 지역코드 컬럼 추가
    df.insert(0, '지역코드', region_code)
    df.insert(0, '지역명', region_name)

    # 컬럼 순서를 expected_columns에 맞추고, 없는 컬럼은 NaN으로 채움
    df = df.reindex(columns=expected_columns)

    all_dataframes.append(df)

print(f"Successfully processed {len(all_dataframes)} CSV files.")

# 3) 여러 DataFrame을 세로로 concat 해서 하나로 만들기
combined_df = pd.concat(all_dataframes, ignore_index=True)

print("Combined shape:", combined_df.shape)
print(combined_df.head())


Found CSV files:
/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023/kdca_2023_week_서울.csv
/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023/kdca_2023_week_경기_안양시_동안구_101.csv
/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023/kdca_2023_week_경기_의왕시_111.csv
/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023/kdca_2023_week_경기_수원시_권선구_093.csv
/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023/kdca_2023_week_경기_부천시_130.csv
/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023/kdca_2023_week_경기_고양시_덕양구_076.csv
/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023/kdca_2023_week_경기_포천시_116.csv
/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023/kdca_2023_week_경기_안성시_100.csv
/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023/kdca_2023_week_경기_고양시_일산동구_077.csv
/content/drive/MyDrive/weat

In [ ]:
import unicodedata
import pandas as pd

def normalize_korean(s):
    if pd.isna(s):
        return s
    # 문자열로 변환 후 NFC로 정규화
    return unicodedata.normalize('NFC', str(s))

In [ ]:
# 2-1) 매핑 규칙 (부분 문자열 매칭용)
region_group_rules_raw = {
    '동두천': [
        '동두천시', '구리시', '부천시_오정구', '과천시', '부천시', '광명시',
        '부천시_소사구', '고양시_덕양구', '의정부시', '고양시_일산동구',
        '고양시_일산서구', '하남시', '남양주시', '김포시', '양주시', '파주시'
    ],
    '파주': [
        '포천시', '연천군'
    ],
    '수원': [
        '수원시_팔달구', '수원시_장안구', '수원시_권선구', '화성시',
        '수원시_영통구', '군포시', '용인시_기흥구', '의왕시',
        '성남시_수정구', '안산시_단원구', '성남시_중원구', '오산시',
        '안산시_상록구', '안양시_동안구', '용인시_수지구', '시흥시',
        '안양시_만안구', '성남시_분당구', '용인시_처인구', '평택시'
    ],
    '양평': [
        '양평군', '광주시', '가평군'
    ],
    '이천': [
        '이천시', '여주시', '안성시'
    ],
    '서울': [
        '서울', '서울특별시'
    ]
}

region_group_rules = {
    base_region: [normalize_korean(kw) for kw in keywords]
    for base_region, keywords in region_group_rules_raw.items()
}

In [ ]:
def map_region_to_base(region_name: str) -> str:
    if pd.isna(region_name):
        return None

    # 지역명도 NFC 정규화
    rn = normalize_korean(region_name)

    for base_region, keywords in region_group_rules.items():
        # 예: '경기_안양시_동안구' 안에 '안양시_동안구'가 들어 있는지 검사
        if any(kw in rn for kw in keywords):
            return base_region

    return None

In [ ]:
# 1) 서울 파일이 combined_df에 들어왔는지
print("서울 관련 지역명:")
print(combined_df[combined_df['지역명'].str.contains('서울', na=False)]
      [['지역명', '지역코드']].drop_duplicates())

# 2) 기준지역 매핑 결과 확인
combined_df['기준지역'] = combined_df['지역명'].apply(map_region_to_base)
print("서울 관련 기준지역 매핑:")
print(combined_df[combined_df['지역명'].str.contains('서울', na=False)]
      [['지역명', '기준지역']].drop_duplicates())

# 3) 최종 final_df에서 지역명 확인
print("최종 지역명 목록:", final_df['지역명'].unique())

서울 관련 지역명:
Empty DataFrame
Columns: [지역명, 지역코드]
Index: []
서울 관련 기준지역 매핑:
Empty DataFrame
Columns: [지역명, 기준지역]
Index: []
최종 지역명 목록: ['동두천' '수원' '양평' '이천' '파주']


In [ ]:
sum_columns = [f'COLUMN{i}' for i in range(1, 54)]


In [ ]:
for col in sum_columns:
    mapped_df[col] = pd.to_numeric(mapped_df[col], errors='coerce')


In [ ]:
# 집계 규칙 정의
agg_dict = {col: 'sum' for col in sum_columns}

# TITLE은 같은 질병이면 대부분 동일할 가능성이 높으므로 'first' 사용
agg_dict['TITLE'] = 'first'

# 기준: 기준지역 + SUBTITLE(질병명)
grouped_df = mapped_df.groupby(
    ['기준지역', 'SUBTITLE'],
    as_index=False
).agg(agg_dict)

print(grouped_df.head())


  기준지역 SUBTITLE  COLUMN1  COLUMN2  COLUMN3  COLUMN4  COLUMN5  COLUMN6  \
0  동두천      뎅기열       25        0        0        1        0        0   
1  동두천      라임병        4        0        0        0        0        0   
2  동두천   레지오넬라증       51        0        3        2        0        0   
3  동두천     말라리아      350        0        0        0        0        0   
4  동두천  비브리오패혈증        3        0        0        0        0        0   

   COLUMN7  COLUMN8  ...  COLUMN45  COLUMN46  COLUMN47  COLUMN48  COLUMN49  \
0        1        0  ...         1         1         1         0         0   
1        0        0  ...         0         0         0         0         0   
2        2        0  ...         1         0         0         1         2   
3        1        0  ...         1         1         0         0         0   
4        0        0  ...         0         0         0         0         0   

   COLUMN50  COLUMN51  COLUMN52  COLUMN53  TITLE  
0         0         0         0         2

In [ ]:
# 예시: 기준지역별 대표 코드 (실제 ASOS 코드로 교체하세요)
base_region_code_map = {
    '동두천': '98',
    '파주':   '99',
    '수원':   '119',
    '양평':   '202',
    '이천':   '203',
    '서울':   '108',
}


In [ ]:
# 5-1) 최종 지역명/지역코드 컬럼 생성
grouped_df['지역명'] = grouped_df['기준지역']
grouped_df['지역코드'] = grouped_df['기준지역'].map(base_region_code_map)

# 5-2) 최종 컬럼 순서 정의
final_columns = (
    ['지역명', '지역코드'] +
    [f'COLUMN{i}' for i in range(1, 54)] +
    ['TITLE', 'SUBTITLE']
)

# 혹시 기준지역 컬럼은 최종에는 필요 없으니 제거
final_df = grouped_df.reindex(columns=final_columns)

print(final_df.head())

# 5-3) CSV로 저장
output_file_path = '/content/combined_kdca_2023_경기_6regions.csv'
final_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"Final aggregated file saved to: {output_file_path}")


   지역명 지역코드  COLUMN1  COLUMN2  COLUMN3  COLUMN4  COLUMN5  COLUMN6  COLUMN7  \
0  동두천   98       25        0        0        1        0        0        1   
1  동두천   98        4        0        0        0        0        0        0   
2  동두천   98       51        0        3        2        0        0        2   
3  동두천   98      350        0        0        0        0        0        1   
4  동두천   98        3        0        0        0        0        0        0   

   COLUMN8  ...  COLUMN46  COLUMN47  COLUMN48  COLUMN49  COLUMN50  COLUMN51  \
0        0  ...         1         1         0         0         0         0   
1        0  ...         0         0         0         0         0         0   
2        0  ...         0         0         1         2         1         1   
3        0  ...         1         0         0         0         0         0   
4        0  ...         0         0         0         0         0         0   

   COLUMN52  COLUMN53  TITLE  SUBTITLE  
0         0    

In [ ]:
# 정렬용

import pandas as pd

# 🔹 1) 기존 최종 파일 불러오기
input_file = '/content/drive/MyDrive/weather-sales-forecast-ai/combined_kdca_2023_경기_6regions.csv'
df = pd.read_csv(input_file)

# 🔹 2) 질병 순서를 원래 정의한 순서로 만들기
disease_order = [
    '장티푸스',
    '파라티푸스',
    '세균성이질',
    '한센병',
    '말라리아',
    '레지오넬라증',
    '비브리오패혈증',
    '쯔쯔가무시증',
    '신증후군출혈열',
    '뎅기열',
    '라임병',
    '중증열성혈소판감소증후군(SFTS)',
    '지카바이러스감염증',
]

# SUBTITLE을 **카테고리형**으로 지정 (정렬 순서 고정)
df['SUBTITLE'] = pd.Categorical(
    df['SUBTITLE'],
    categories=disease_order,
    ordered=True
)

# 🔹 3) 지역별 → SUBTITLE 순서대로 정렬
df_sorted = df.sort_values(
    by=['지역명', 'SUBTITLE']
).reset_index(drop=True)

# 🔹 4) 정렬 결과 저장
output_file = '/content/combined_kdca_2023_경기_6regions_sorted.csv'
df_sorted.to_csv(output_file, index=False, encoding='utf-8-sig')

print("📌 SUBTITLE 순서 정렬 완료!")
print(f"📁 저장 경로: {output_file}")

# 🔹 5) 앞부분 확인
print(df_sorted.head(20))


📌 SUBTITLE 순서 정렬 완료!
📁 저장 경로: /content/combined_kdca_2023_경기_6regions_sorted.csv
    지역명  지역코드  COLUMN1  COLUMN2  COLUMN3  COLUMN4  COLUMN5  COLUMN6  COLUMN7  \
0   동두천    98        1        0        0        0        0        0        0   
1   동두천    98        3        0        0        0        0        0        0   
2   동두천    98        4        0        0        0        0        0        0   
3   동두천    98        0        0        0        0        0        0        0   
4   동두천    98      350        0        0        0        0        0        1   
5   동두천    98       51        0        3        2        0        0        2   
6   동두천    98        3        0        0        0        0        0        0   
7   동두천    98       64        0        0        0        0        0        0   
8   동두천    98       17        0        0        0        0        0        0   
9   동두천    98       25        0        0        1        0        0        1   
10  동두천    98        4        0        

---

In [3]:
import os
import pandas as pd
import unicodedata

# ============================
# 0. 공통 설정
# ============================

# 연도별 원본 CSV 폴더 (연도만 바뀐다고 가정)
BASE_DIR = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터'

YEARS = [2023, 2024, 2025]

# 기대 컬럼 (공통)
SUM_COLUMNS = [f'COLUMN{i}' for i in range(1, 54)]
EXPECTED_COLUMNS = ['지역명', '지역코드'] + SUM_COLUMNS + ['TITLE', 'SUBTITLE']

# 질병명 정렬 순서 (원본 순서)
DISEASE_ORDER = [
    '장티푸스',
    '파라티푸스',
    '세균성이질',
    '한센병',
    '말라리아',
    '레지오넬라증',
    '비브리오패혈증',
    '쯔쯔가무시증',
    '신증후군출혈열',
    '뎅기열',
    '라임병',
    '중증열성혈소판감소증후군(SFTS)',
    '지카바이러스감염증',
]

# 기준지역 → ASOS 코드 (연도와 무관, 공통 사용)
BASE_REGION_CODE_MAP = {
    '동두천': '98',
    '파주':   '99',
    '수원':   '119',
    '양평':   '202',
    '이천':   '203',
    '서울':   '108',
}

# ============================
# 1. 유틸 함수 정의
# ============================

def normalize_korean(s):
    """한글 문자열을 NFC로 정규화 (유니코드 조합형/완성형 차이 방지)."""
    if pd.isna(s):
        return s
    return unicodedata.normalize('NFC', str(s))


def parse_region_from_filename(filename: str):
    """
    파일명에서 지역명 / 지역코드 추출.
    예)
      - kdca_2023_week_경기_안양시_동안구_101.csv
      - kdca_2023_week_서울.csv
    두 패턴 모두 처리해서 (지역명, 지역코드)를 리턴.
    """
    base = filename.replace('.csv', '')
    parts = base.split('_')

    # 최소 길이 방어
    if len(parts) < 4:
        return None, None

    # 서울처럼 코드 없이 지역명만 있는 경우: kdca_2023_week_서울
    if len(parts) == 4:
        region_name = parts[3]      # '서울'
        region_code = None          # 코드 없음 (기준지역 코드 매핑에서 채움)
    else:
        # 일반 패턴: 마지막은 코드, 중간은 지역명
        region_code = parts[-1]
        region_name_parts = parts[3:-1]
        region_name = '_'.join(region_name_parts)  # 예: '경기_안양시_동안구'

    # 한글 정규화까지 바로 적용
    return normalize_korean(region_name), region_code


# 기준지역 매핑 규칙 (부분 문자열 매칭용)
REGION_GROUP_RULES_RAW = {
    '동두천': [
        '동두천시', '구리시', '부천시_오정구', '과천시', '부천시', '광명시',
        '부천시_소사구', '고양시_덕양구', '의정부시', '고양시_일산동구',
        '고양시_일산서구', '하남시', '남양주시', '김포시', '양주시', '파주시'
    ],
    '파주': ['포천시', '연천군'],
    '수원': [
        '수원시_팔달구', '수원시_장안구', '수원시_권선구', '화성시',
        '수원시_영통구', '군포시', '용인시_기흥구', '의왕시',
        '성남시_수정구', '안산시_단원구', '성남시_중원구', '오산시',
        '안산시_상록구', '안양시_동안구', '용인시_수지구', '시흥시',
        '안양시_만안구', '성남시_분당구', '용인시_처인구', '평택시'
    ],
    '양평': ['양평군', '광주시', '가평군'],
    '이천': ['이천시', '여주시', '안성시'],
    '서울': ['서울', '서울특별시', '서울시'],
}

# 규칙 내부도 NFC로 정규화해서 매칭 안정성 확보
REGION_GROUP_RULES = {
    base_region: [normalize_korean(kw) for kw in keywords]
    for base_region, keywords in REGION_GROUP_RULES_RAW.items()
}


def map_region_to_base(region_name: str) -> str:
    """
    행정구 단위 지역명을 6개 기준지역으로 매핑.
    매칭 실패 시 None 리턴.
    """
    if pd.isna(region_name):
        return None

    rn = normalize_korean(region_name)

    for base_region, keywords in REGION_GROUP_RULES.items():
        if any(kw in rn for kw in keywords):
            return base_region

    return None


# ============================
# 2. 연도별 루프 처리
# ============================

for year in YEARS:
    csv_directory = os.path.join(BASE_DIR, str(year))
    print(f"\n==============================")
    print(f"📆 {year}년 처리 시작 (디렉토리: {csv_directory})")

    # 2-1) 폴더 내 모든 CSV 파일 경로 수집 (하위 폴더까지 포함하고 싶으면 os.walk 사용)
    csv_files = []
    for root, dirs, files in os.walk(csv_directory):
        for fname in files:
            if fname.lower().endswith('.csv'):
                csv_files.append(os.path.join(root, fname))

    print(f"  ▷ 발견된 CSV 파일 수: {len(csv_files)}")

    if not csv_files:
        print(f"  ⚠ {year}년 폴더에 CSV 파일이 없습니다. 건너뜁니다.")
        continue

    # 2-2) 각 CSV를 읽어서 공통 스키마로 맞추기
    all_dfs = []

    for file_path in csv_files:
        filename = os.path.basename(file_path)
        region_name, region_code = parse_region_from_filename(filename)

        df = pd.read_csv(file_path)

        # 앞에 지역코드, 지역명 컬럼 추가
        df.insert(0, '지역코드', region_code)
        df.insert(0, '지역명', region_name)

        # 기대하는 컬럼 순서 강제 (없는 컬럼은 NaN으로 채워짐)
        df = df.reindex(columns=EXPECTED_COLUMNS)

        all_dfs.append(df)

    # 2-3) 연도별 전체 통합
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"  ▷ 통합 후 shape: {combined_df.shape}")

    # 2-4) 기준지역 매핑
    combined_df['기준지역'] = combined_df['지역명'].apply(map_region_to_base)
    print("  ▷ 기준지역 매핑 예시:")
    print(combined_df[['지역명', '기준지역']].drop_duplicates().head(10))

    # 기준지역 없는 행 제거 (필요하면 따로 로그 확인 가능)
    mapped_df = combined_df.dropna(subset=['기준지역']).copy()
    print(f"  ▷ 기준지역 매핑 후 남은 행 수: {mapped_df.shape[0]}")

    # 2-5) 이 연도에서 실제 데이터가 있는 주차 컬럼만 선택
    sum_cols_year = [
        col for col in SUM_COLUMNS
        if col in mapped_df.columns and not mapped_df[col].isna().all()
    ]
    print(f"  ▷ {year}년 사용 주차 컬럼 수: {len(sum_cols_year)}")
    print(f"    예시 주차 컬럼: {sum_cols_year[:10]}")

    # 2-6) 수치 컬럼 숫자형 변환 (해당 연도에서 사용하는 컬럼만)
    for col in sum_cols_year:
        mapped_df[col] = pd.to_numeric(mapped_df[col], errors='coerce')

    # 2-7) 기준지역 + SUBTITLE(질병명) 기준 그룹 집계
    agg_dict = {col: 'sum' for col in sum_cols_year}
    agg_dict['TITLE'] = 'first'

    grouped_df = mapped_df.groupby(
        ['기준지역', 'SUBTITLE'],
        as_index=False,
        sort=False  # 등장 순서 유지 (나중에 우리가 직접 정렬)
    ).agg(agg_dict)

    # 2-8) 최종 지역명/지역코드 컬럼 생성
    grouped_df['지역명'] = grouped_df['기준지역']
    grouped_df['지역코드'] = grouped_df['기준지역'].map(BASE_REGION_CODE_MAP)

    # 2-9) SUBTITLE(질병명) 순서를 원래 정의대로 강제
    grouped_df['SUBTITLE'] = pd.Categorical(
        grouped_df['SUBTITLE'],
        categories=DISEASE_ORDER,
        ordered=True
    )

    # 2-10) 지역명 → SUBTITLE 순서대로 정렬
    final_df = grouped_df.sort_values(
        by=['지역명', 'SUBTITLE']
    ).reset_index(drop=True)

    # 2-11) 최종 컬럼 순서 강제
    FINAL_COLUMNS = ['지역명', '지역코드'] + sum_cols_year + ['TITLE', 'SUBTITLE']
    final_df = final_df.reindex(columns=FINAL_COLUMNS)

    # 2-12) 저장
    output_file = os.path.join(
        BASE_DIR,
        f'combined_kdca_{year}_경기_6regions_sorted.csv'
    )
    final_df.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"  ✅ {year}년 처리 완료. 저장 경로: {output_file}")
    print(final_df.head(5))

print("\n🎉 모든 연도(2023, 2024, 2025) 처리 완료!")



📆 2023년 처리 시작 (디렉토리: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/2023)
  ▷ 발견된 CSV 파일 수: 45
  ▷ 통합 후 shape: (585, 57)
  ▷ 기준지역 매핑 예시:
             지역명 기준지역
0             서울   서울
13    경기_안양시_동안구   수원
26        경기_의왕시   수원
39    경기_수원시_권선구   수원
52        경기_부천시  동두천
65    경기_고양시_덕양구  동두천
78        경기_포천시   파주
91        경기_안성시   이천
104  경기_고양시_일산동구  동두천
117      경기_의정부시  동두천
  ▷ 기준지역 매핑 후 남은 행 수: 585
  ▷ 2023년 사용 주차 컬럼 수: 53
    예시 주차 컬럼: ['COLUMN1', 'COLUMN2', 'COLUMN3', 'COLUMN4', 'COLUMN5', 'COLUMN6', 'COLUMN7', 'COLUMN8', 'COLUMN9', 'COLUMN10']
  ✅ 2023년 처리 완료. 저장 경로: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/combined_kdca_2023_경기_6regions_sorted.csv
   지역명 지역코드  COLUMN1  COLUMN2  COLUMN3  COLUMN4  COLUMN5  COLUMN6  COLUMN7  \
0  동두천   98        1        0        0        0        0        0        0   
1  동두천   98        3        0        0        0        0        0        0   
2  동두천   98        4        0        0        0        0        0   

---

2023·2024·2025 combined 파일 → 주별 long 포맷으로 변환 코드

In [5]:
import os
import pandas as pd
from datetime import date, timedelta

# =========================================
# 0. 설정: 파일 경로 / 컬럼 정의
# =========================================

BASE_DIR = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터'

# 앞에서 만든 연도별 combined 파일들
YEAR_FILES = {
    2023: os.path.join(BASE_DIR, 'combined_kdca_2023_경기_6regions_sorted.csv'),
    2024: os.path.join(BASE_DIR, 'combined_kdca_2024_경기_6regions_sorted.csv'),
    2025: os.path.join(BASE_DIR, 'combined_kdca_2025_경기_6regions_sorted.csv'),
}

# COLUMN1 = 총건수, COLUMN2~53 = 1~52주
TOTAL_COL = 'COLUMN1'
WEEK_COLS = [f'COLUMN{i}' for i in range(2, 54)]  # 2~53

# =========================================
# 1. 주차 → 주시작일 / 주종료일 계산 함수
# =========================================

def get_week_start_end(year: int, week_no: int):
    """
    해당 year의 week_no(1~52)에 대한 주 시작일(일요일)과 종료일(토요일)을 반환.
    - 1주는 '해당 연도의 1월1일이 속한 주의 일요일~토요일'
    """
    jan1 = date(year, 1, 1)
    # 월요일=0, ..., 일요일=6. 1월1일이 속한 주의 '일요일'을 구한다.
    days_since_sunday = (jan1.weekday() + 1) % 7
    week1_sunday = jan1 - timedelta(days=days_since_sunday)

    # 해당 주차의 일요일 = 1주차 일요일 + (week_no-1)*7일
    sunday = week1_sunday + timedelta(weeks=week_no - 1)
    saturday = sunday + timedelta(days=6)

    return sunday, saturday

# =========================================
# 2. wide → long 변환 + 주차/날짜 붙이기 함수
# =========================================

def make_weekly_long_df(year: int, input_path: str) -> pd.DataFrame:
    """
    연도별 combined_kdca_YYYY_경기_6regions_sorted.csv 를
    주별 long 포맷으로 변환.
    반환 컬럼:
      지점명, 지점코드, 질병명, 질병총합, 년도, 주차, 주시작일, 주종료일, 주별발생건수
    """
    df = pd.read_csv(input_path)

    # 1) 기본 컬럼 rename (이름 통일)
    df = df.rename(columns={
        '지역명': '지점명',
        '지역코드': '지점코드',
        'SUBTITLE': '질병명',
        TOTAL_COL: '질병총합',
    })

    # 2) 연도 컬럼 추가
    df['년도'] = year

    # 3) 이 연도에서 실제로 사용할 주차 컬럼 동적으로 선정
    #    - COLUMN으로 시작
    #    - COLUMN1(총합)은 제외
    #    - 컬럼이 존재하고, 전부 NaN이 아닌 것만
    week_cols = []
    for col in df.columns:
        if col.startswith('COLUMN') and col != TOTAL_COL:
            if not df[col].isna().all():
                week_cols.append(col)

    # 숫자 기준으로 정렬 (COLUMN2, COLUMN3, ...)
    week_cols = sorted(week_cols, key=lambda c: int(c.replace('COLUMN', '')))
    print(f"[{year}] 사용 주차 컬럼:", week_cols)

    # 4) wide → long : 실제 존재하는 주차 컬럼만 melt
    long_df = df.melt(
        id_vars=['지점명', '지점코드', '질병명', '질병총합', '년도'],
        value_vars=week_cols,
        var_name='week_col',
        value_name='주별발생건수'
    )

    # 5) week_col → 주차 번호 (COLUMN2 → 1주, COLUMN3 → 2주, ...)
    #    COLUMN숫자 - 1 = 주차
    long_df['col_idx'] = long_df['week_col'].str.extract(r'(\d+)').astype(int)
    long_df['주차'] = long_df['col_idx'] - 1

    # 주별발생건수 NaN → 0으로 바꾸고 싶으면 여기서 처리 (선택)
    # long_df['주별발생건수'] = long_df['주별발생건수'].fillna(0)

    # 6) 주차 정보(1~52)에 대해 주시작일/주종료일 사전 만들기
    # 실제 사용된 주차 범위만큼 week_df 생성
    max_week = long_df['주차'].max()
    week_info = []
    for w in range(1, max_week + 1):  # 1~max_week
        ws, we = get_week_start_end(year, w)
        week_info.append({
            '주차': w,
            '주시작일': ws,
            '주종료일': we,
        })
    week_df = pd.DataFrame(week_info)

    # 7) long_df와 week_df를 주차 기준으로 merge
    long_df = long_df.merge(week_df, on='주차', how='left')

    # 8) 날짜를 문자열(YYYY-MM-DD)로 저장하고 싶다면:
    long_df['주시작일'] = long_df['주시작일'].astype(str)
    long_df['주종료일'] = long_df['주종료일'].astype(str)

    # 9) 최종 컬럼 순서 정리
    final_cols = [
        '지점명',
        '지점코드',
        '질병명',
        '질병총합',
        '년도',
        '주차',
        '주시작일',
        '주종료일',
        '주별발생건수',  # 모델링/분석에 필요한 실제 target 값
    ]
    long_df = long_df[final_cols].sort_values(
        by=['지점명', '질병명', '년도', '주차']
    ).reset_index(drop=True)

    return long_df

# =========================================
# 3. 연도별로 돌리고, 각각 저장
# =========================================

yearly_long_dfs = []

for year, path in YEAR_FILES.items():
    if not os.path.exists(path):
        print(f"⚠ {year}년 파일이 없습니다: {path}")
        continue

    print(f"📆 {year}년 변환 중: {path}")
    long_df = make_weekly_long_df(year, path)

    # 연도별 개별 파일로도 저장
    out_path_year = os.path.join(
        BASE_DIR,
        f'kdca_weekly_long_{year}_6regions.csv'
    )
    long_df.to_csv(out_path_year, index=False, encoding='utf-8-sig')
    print(f"  ✅ {year}년 주별 long 파일 저장: {out_path_year}")
    print(long_df.head())

    yearly_long_dfs.append(long_df)

# =========================================
# 4. 원하면 3개 연도를 묶어서 하나의 파일로도 저장
# =========================================

if yearly_long_dfs:
    all_long = pd.concat(yearly_long_dfs, ignore_index=True)
    out_path_all = os.path.join(
        BASE_DIR,
        'kdca_weekly_long_2023_2025_6regions_all.csv'
    )
    all_long.to_csv(out_path_all, index=False, encoding='utf-8-sig')
    print("🎉 2023~2025 전체 주별 long 파일 저장:", out_path_all)
    print(all_long.head())


📆 2023년 변환 중: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/combined_kdca_2023_경기_6regions_sorted.csv
[2023] 사용 주차 컬럼: ['COLUMN2', 'COLUMN3', 'COLUMN4', 'COLUMN5', 'COLUMN6', 'COLUMN7', 'COLUMN8', 'COLUMN9', 'COLUMN10', 'COLUMN11', 'COLUMN12', 'COLUMN13', 'COLUMN14', 'COLUMN15', 'COLUMN16', 'COLUMN17', 'COLUMN18', 'COLUMN19', 'COLUMN20', 'COLUMN21', 'COLUMN22', 'COLUMN23', 'COLUMN24', 'COLUMN25', 'COLUMN26', 'COLUMN27', 'COLUMN28', 'COLUMN29', 'COLUMN30', 'COLUMN31', 'COLUMN32', 'COLUMN33', 'COLUMN34', 'COLUMN35', 'COLUMN36', 'COLUMN37', 'COLUMN38', 'COLUMN39', 'COLUMN40', 'COLUMN41', 'COLUMN42', 'COLUMN43', 'COLUMN44', 'COLUMN45', 'COLUMN46', 'COLUMN47', 'COLUMN48', 'COLUMN49', 'COLUMN50', 'COLUMN51', 'COLUMN52', 'COLUMN53']
  ✅ 2023년 주별 long 파일 저장: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/kdca_weekly_long_2023_6regions.csv
   지점명  지점코드  질병명  질병총합    년도  주차        주시작일        주종료일  주별발생건수
0  동두천    98  뎅기열    25  2023   1  2023-01-01  2023-01-07       0
1 

---

주간 long 파일 → 일별 데이터로 확장하는 전체 코드

In [6]:
import os
import pandas as pd

# =========================================
# 0. 경로 및 대상 파일 설정
# =========================================

BASE_DIR = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터'

# ✔ 연도별 KDCA 주간 long 파일 사용 (파일명 정정 완료)
YEARLY_WEEKLY_FILES = {
    2023: os.path.join(BASE_DIR, 'kdca_weekly_long_2023_6regions.csv'),
    2024: os.path.join(BASE_DIR, 'kdca_weekly_long_2024_6regions.csv'),
    2025: os.path.join(BASE_DIR, 'kdca_weekly_long_2025_6regions.csv'),
}

# 결과 파일 저장 경로 (각 연도별)
def output_daily_path(year):
    return os.path.join(BASE_DIR, f'kdca_daily_{year}_6regions.csv')

# =========================================
# 1. 주간 → 일별 변환 함수
# =========================================
from datetime import timedelta

def weekly_to_daily(df: pd.DataFrame) -> pd.DataFrame:
    # 주시작일/주종료일 datetime 변환
    df['주시작일'] = pd.to_datetime(df['주시작일'])
    df['주종료일'] = pd.to_datetime(df['주종료일'])

    # 일자 리스트 생성 (일~토, 7일)
    df['날짜리스트'] = df.apply(
        lambda row: pd.date_range(start=row['주시작일'], end=row['주종료일'], freq='D'),
        axis=1
    )

    # explode로 주차 1행 → 7행 확장
    daily_df = df.explode('날짜리스트').reset_index(drop=True)
    daily_df = daily_df.rename(columns={'날짜리스트': '날짜'})

    # 일자 파생 변수
    daily_df['년도_from_date'] = daily_df['날짜'].dt.year
    daily_df['월'] = daily_df['날짜'].dt.month
    daily_df['일'] = daily_df['날짜'].dt.day

    # 일별 발생건수 = 주별발생건수 / 7
    daily_df['주별발생건수'] = pd.to_numeric(daily_df['주별발생건수'], errors='coerce')
    daily_df['일별발생건수'] = daily_df['주별발생건수'] / 7.0

    # 최종 컬럼 정렬
    final_columns = [
        '지점명', '지점코드', '질병명', '질병총합',
        '년도', '주차',
        '날짜', '주시작일', '주종료일',
        '주별발생건수', '일별발생건수',
        '년도_from_date', '월', '일'
    ]
    final_columns = [c for c in final_columns if c in daily_df.columns]
    daily_df = daily_df[final_columns].sort_values(
        by=['지점명', '질병명', '날짜']
    ).reset_index(drop=True)

    return daily_df

# =========================================
# 2. 각 연도별로 처리 + 저장
# =========================================

for year, weekly_file in YEARLY_WEEKLY_FILES.items():
    if not os.path.exists(weekly_file):
        print(f"⚠ {year}년 파일 없음 → 스킵: {weekly_file}")
        continue

    print(f"\n📆 {year}년 주간 KDCA 파일 로드: {weekly_file}")
    weekly_df = pd.read_csv(weekly_file)
    print("  샘플 확인:")
    print(weekly_df.head())

    daily_df = weekly_to_daily(weekly_df)

    out_path = output_daily_path(year)
    daily_df.to_csv(out_path, index=False, encoding='utf-8-sig')

    print(f"  ✅ {year}년 → 일별 데이터 저장 완료: {out_path}")
    print(daily_df.head(10))

print("\n🎉 모든 연도(2023, 2024, 2025) KDCA 일별 데이터 생성 완료!")



📆 2023년 주간 KDCA 파일 로드: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/kdca_weekly_long_2023_6regions.csv
  샘플 확인:
   지점명  지점코드  질병명  질병총합    년도  주차        주시작일        주종료일  주별발생건수
0  동두천    98  뎅기열    25  2023   1  2023-01-01  2023-01-07       0
1  동두천    98  뎅기열    25  2023   2  2023-01-08  2023-01-14       0
2  동두천    98  뎅기열    25  2023   3  2023-01-15  2023-01-21       1
3  동두천    98  뎅기열    25  2023   4  2023-01-22  2023-01-28       0
4  동두천    98  뎅기열    25  2023   5  2023-01-29  2023-02-04       0
  ✅ 2023년 → 일별 데이터 저장 완료: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/kdca_daily_2023_6regions.csv
   지점명  지점코드  질병명  질병총합    년도  주차         날짜       주시작일       주종료일  주별발생건수  \
0  동두천    98  뎅기열    25  2023   1 2023-01-01 2023-01-01 2023-01-07       0   
1  동두천    98  뎅기열    25  2023   1 2023-01-02 2023-01-01 2023-01-07       0   
2  동두천    98  뎅기열    25  2023   1 2023-01-03 2023-01-01 2023-01-07       0   
3  동두천    98  뎅기열    25  2023   1 2023-01-04 2023-01-

---

ASOS 일별 데이터에서 실제로 질병 발생에 영향을 주는 컬럼 선정

In [7]:
import pandas as pd

asos_path = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/OBS_ASOS_DD.csv'
selected_columns = [
    '지점','지점명','일시',
    '평균기온(°C)','최저기온(°C)','최고기온(°C)',
    '평균 이슬점온도(°C)','평균 상대습도(%)','최소 상대습도(%)','평균 증기압(hPa)',
    '일강수량(mm)','강수 계속시간(hr)','1시간 최다강수량(mm)',
    '평균 풍속(m/s)','최대 순간 풍속(m/s)',
    '합계 일조시간(hr)','합계 일사량(MJ/m2)',
    '평균 지면온도(°C)','안개 계속시간(hr)'
]

asos_df = pd.read_csv(asos_path, encoding="cp949")

# 필요한 컬럼만 선택
asos_df = asos_df[selected_columns].copy()

# 일시(datetime 형변환)
asos_df['일시'] = pd.to_datetime(asos_df['일시'])

print(asos_df.head())


   지점  지점명         일시  평균기온(°C)  최저기온(°C)  최고기온(°C)  평균 이슬점온도(°C)  평균 상대습도(%)  \
0  98  동두천 2023-01-01      -2.6      -6.5       3.8         -11.9        56.0   
1  98  동두천 2023-01-02      -6.8     -11.6      -0.3         -15.6        52.0   
2  98  동두천 2023-01-03      -7.6     -13.5      -0.5         -16.0        55.0   
3  98  동두천 2023-01-04      -5.0     -11.4       2.8         -13.5        55.0   
4  98  동두천 2023-01-05      -4.4     -10.7       4.0         -12.0        59.8   

   최소 상대습도(%)  평균 증기압(hPa)  일강수량(mm)  강수 계속시간(hr)  1시간 최다강수량(mm)  평균 풍속(m/s)  \
0        21.8          2.7       NaN          NaN            NaN         1.1   
1        26.2          1.8       NaN          NaN            NaN         0.6   
2        24.7          1.8       NaN          NaN            NaN         0.5   
3        21.7          2.2       NaN          NaN            NaN         0.3   
4        21.0          2.5       NaN          NaN            NaN         0.3   

   최대 순간 풍속(m/s)  합계 일조시간(hr)  합

KDCA 일별(3개 연도) 통합 + ASOS 필요한 컬럼만 선택 + merge 코드

In [8]:
import os
import pandas as pd

# =========================================
# 0. 경로 및 파일 설정
# =========================================

BASE_DIR = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터'

KDCA_DAILY_FILES = [
    os.path.join(BASE_DIR, 'kdca_daily_2023_6regions.csv'),
    os.path.join(BASE_DIR, 'kdca_daily_2024_6regions.csv'),
    os.path.join(BASE_DIR, 'kdca_daily_2025_6regions.csv'),
]

ASOS_PATH = os.path.join(BASE_DIR, 'OBS_ASOS_DD.csv')

# ASOS에서 사용할 컬럼들
ASOS_SELECTED_COLS = [
    '지점',            # ASOS 지점코드 (KDCA 지점코드와 매칭)
    '지점명',
    '일시',            # 날짜

    # 🌡 온도
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',

    # 💧 습도/증기압
    '평균 이슬점온도(°C)',
    '평균 상대습도(%)',
    '최소 상대습도(%)',
    '평균 증기압(hPa)',

    # ☔ 강수 관련
    '일강수량(mm)',
    '강수 계속시간(hr)',
    '1시간 최다강수량(mm)',

    # 💨 풍속 관련
    '평균 풍속(m/s)',
    '최대 순간 풍속(m/s)',

    # ☀ 일사 관련
    '합계 일조시간(hr)',
    '합계 일사량(MJ/m2)',

    # 🌫 지면/안개
    '평균 지면온도(°C)',
    '안개 계속시간(hr)',
]

# =========================================
# 1. KDCA 일별 3개 연도 파일 통합
# =========================================

kdca_dfs = []
for path in KDCA_DAILY_FILES:
    if os.path.exists(path):
        df = pd.read_csv(path)
        kdca_dfs.append(df)
        print(f"✅ KDCA daily loaded: {path}, shape={df.shape}")
    else:
        print(f"⚠ KDCA daily file not found: {path}")

kdca_all = pd.concat(kdca_dfs, ignore_index=True)

# 날짜 컬럼을 datetime으로 변환
kdca_all['날짜'] = pd.to_datetime(kdca_all['날짜'])

print("📌 KDCA all years combined:", kdca_all.shape)
print(kdca_all.head())

# =========================================
# 2. ASOS 기후데이터에서 필요한 컬럼만 선택
# =========================================

asos = pd.read_csv(ASOS_PATH, encoding="cp949")

# 필요한 컬럼만 남기기
asos = asos[ASOS_SELECTED_COLS].copy()

# 일시를 datetime으로 변환
asos['일시'] = pd.to_datetime(asos['일시'])

print("📌 ASOS selected columns shape:", asos.shape)
print(asos.head())

# =========================================
# 3. KDCA × ASOS merge (지점코드 + 날짜 기준)
# =========================================

merged = pd.merge(
    kdca_all,
    asos,
    how='left',
    left_on=['지점코드', '날짜'],  # KDCA 키
    right_on=['지점', '일시'],     # ASOS 키
)

print("📌 merged shape:", merged.shape)
print(merged.head())

# =========================================
# 3-1. merge 후 중복 컬럼 정리
# =========================================

# ASOS 중복 키 컬럼 제거
merged = merged.drop(columns=['지점', '일시'], errors='ignore')

# merge 후 생긴 지점명_x / 지점명_y 정리
if '지점명_x' in merged.columns:
    merged = merged.rename(columns={'지점명_x': '지점명'})
if '지점명_y' in merged.columns:
    merged = merged.drop(columns=['지점명_y'], errors='ignore')

# 제거 후 결과 확인
print("\n📌 After removing duplicated columns:")
print(merged.head())

# =========================================
# 4. 결과 저장
# =========================================

OUTPUT_PATH = os.path.join(BASE_DIR, 'kdca_asos_merged_2023_2025_raw.csv')
merged.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')

print(f"\n✅ KDCA 3개 연도 + ASOS 일별 기후데이터 결합 파일 저장 완료: {OUTPUT_PATH}")


✅ KDCA daily loaded: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/kdca_daily_2023_6regions.csv, shape=(28392, 14)
✅ KDCA daily loaded: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/kdca_daily_2024_6regions.csv, shape=(28392, 14)
✅ KDCA daily loaded: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/kdca_daily_2025_6regions.csv, shape=(26754, 14)
📌 KDCA all years combined: (83538, 14)
   지점명  지점코드  질병명  질병총합    년도  주차         날짜        주시작일        주종료일  주별발생건수  \
0  동두천    98  뎅기열    25  2023   1 2023-01-01  2023-01-01  2023-01-07       0   
1  동두천    98  뎅기열    25  2023   1 2023-01-02  2023-01-01  2023-01-07       0   
2  동두천    98  뎅기열    25  2023   1 2023-01-03  2023-01-01  2023-01-07       0   
3  동두천    98  뎅기열    25  2023   1 2023-01-04  2023-01-01  2023-01-07       0   
4  동두천    98  뎅기열    25  2023   1 2023-01-05  2023-01-01  2023-01-07       0   

   일별발생건수  년도_from_date  월  일  
0     0.0          2023  1  1  
1     0.0          2023  1

---

In [ ]:
# 참고

In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# =========================================
# 0. 경로 및 파일 설정
# =========================================

BASE_DIR = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터'

# (이미 만든) 연도별 KDCA 일별 파일들
KDCA_DAILY_FILES = [
    os.path.join(BASE_DIR, 'kdca_daily_2023_6regions.csv'),
    os.path.join(BASE_DIR, 'kdca_daily_2024_6regions.csv'),
    os.path.join(BASE_DIR, 'kdca_daily_2025_6regions.csv'),
]

# ASOS 원본 파일
ASOS_PATH = os.path.join(BASE_DIR, 'OBS_ASOS_DD.csv')

# ASOS에서 쓸 컬럼들 (앞에서 고른 것)
ASOS_SELECTED_COLS = [
    '지점',            # ASOS 지점코드 (질병데이터 지점코드와 merge)
    '지점명',
    '일시',            # 날짜

    # 🌡 온도
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',

    # 💧 습도 / 증기압
    '평균 이슬점온도(°C)',
    '평균 상대습도(%)',
    '최소 상대습도(%)',
    '평균 증기압(hPa)',

    # ☔ 강수 관련
    '일강수량(mm)',
    '강수 계속시간(hr)',
    '1시간 최다강수량(mm)',

    # 💨 풍속 관련
    '평균 풍속(m/s)',
    '최대 순간 풍속(m/s)',

    # ☀ 일사 관련
    '합계 일조시간(hr)',
    '합계 일사량(MJ/m2)',

    # ❄️ 지면온도
    '평균 지면온도(°C)',

    # 🌫 안개
    '안개 계속시간(hr)',
]

# =========================================
# 1. KDCA 일별 질병 데이터 로드 & 통합
#    (지점명, 지점코드, 질병명, 질병총합, 년도, 주차, 날짜, 주시작일, 주종료일, 주별발생건수, 일별발생건수 ...)
# =========================================

kdca_daily_list = []
for path in KDCA_DAILY_FILES:
    if os.path.exists(path):
        df = pd.read_csv(path)
        kdca_daily_list.append(df)
        print(f"✅ KDCA daily loaded: {path}, shape={df.shape}")
    else:
        print(f"⚠ KDCA daily file not found: {path}")

kdca_daily = pd.concat(kdca_daily_list, ignore_index=True)
kdca_daily['날짜'] = pd.to_datetime(kdca_daily['날짜'])

# 타겟 NaN은 학습에서 의미 없으니 제거
kdca_daily = kdca_daily.dropna(subset=['일별발생건수'])
print("KDCA daily combined:", kdca_daily.shape)

# =========================================
# 2. ASOS 기후 데이터 로드 & 필요한 컬럼만 선택
# =========================================

asos = pd.read_csv(ASOS_PATH, encoding='cp949')

# 필요한 컬럼만
asos = asos[ASOS_SELECTED_COLS].copy()

# 일시를 datetime으로 변환
asos['일시'] = pd.to_datetime(asos['일시'])

print("ASOS raw:", asos.shape)

# =========================================
# 3. KDCA × ASOS merge (지점코드 + 날짜 기준)
# =========================================

# KDCA: 지점코드, 날짜
# ASOS: 지점, 일시

merged = pd.merge(
    kdca_daily,
    asos,
    how='left',
    left_on=['지점코드', '날짜'],
    right_on=['지점', '일시']
)

print("Merged shape:", merged.shape)

# 필요 없는 중복 키/이름 정리
# - KDCA 쪽 지점명은 이미 있고, ASOS 지점명도 같을 수 있음. 하나만 쓰자.
# 여기서는 KDCA 지점명 유지, ASOS 지점명은 참고용으로만 둘 수도 있음.
# 일단 지점, 일시 컬럼은 남겨두되, 지점명은 KDCA 기준으로 사용.
# (원하면 지점명_x, 지점명_y를 따로 정리 가능)

# =========================================
# 4. 결측값 처리 (기후 피처 위주)
# =========================================

# 4-1) 수치형 기후 컬럼 리스트 정의
climate_numeric_cols = [
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',
    '평균 이슬점온도(°C)',
    '평균 상대습도(%)',
    '최소 상대습도(%)',
    '평균 증기압(hPa)',
    '일강수량(mm)',
    '강수 계속시간(hr)',
    '1시간 최다강수량(mm)',
    '평균 풍속(m/s)',
    '최대 순간 풍속(m/s)',
    '합계 일조시간(hr)',
    '합계 일사량(MJ/m2)',
    '평균 지면온도(°C)',
    '안개 계속시간(hr)',
]

# 수치 변환 (문자형 결측 등 방지)
for col in climate_numeric_cols:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')

# 4-2) 결측값 처리 전략
# - 너무 공격적으로 0으로 채우지 않고,
# - 지점코드 단위로 '중앙값'으로 채우는 보수적인 방식 사용.
#   (기상 관측에서 NaN은 '관측 실패'인 경우가 많아서, 0과는 구분하는 게 안전)
# - 만약 강수/일사에서 NaN은 '강수 없음' 의미라는 걸 확신하면, 그때만 0으로 바꾸는 전략도 가능.

for col in climate_numeric_cols:
    merged[col] = merged.groupby('지점코드')[col].transform(
        lambda s: s.fillna(s.median())
    )

# 여전히 남아 있는 NaN은 전체 중앙값으로 한 번 더 방어적으로 처리
for col in climate_numeric_cols:
    if merged[col].isna().any():
        merged[col] = merged[col].fillna(merged[col].median())

print("After NA handling, any NaN in climate columns?",
      merged[climate_numeric_cols].isna().any().any())

# =========================================
# 5. Lag Feature 생성 (기후 → 질병 지연 효과)
# =========================================

# lag를 만들 주요 기후 피처 선택 (너무 많으면 차원 폭발하니 핵심만)
lag_base_cols = [
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',
    '평균 상대습도(%)',
    '일강수량(mm)',
    '합계 일사량(MJ/m2)',
]

# 어떤 lag를 만들지 (일 단위, 1일 / 7일 / 14일 지연)
lags = [1, 7, 14]

merged = merged.sort_values(by=['지점코드', '질병명', '날짜'])

for col in lag_base_cols:
    for lag in lags:
        merged[f'{col}_lag{lag}'] = (
            merged.groupby(['지점코드', '질병명'])[col]
            .shift(lag)
        )

# 5-2) rolling window feature (예: 7일 강수량 합, 7일 평균기온 등)
merged['일강수량_7d_sum'] = (
    merged.groupby(['지점코드', '질병명'])['일강수량(mm)']
    .transform(lambda s: s.rolling(window=7, min_periods=1).sum())
)

merged['평균기온_7d_mean'] = (
    merged.groupby(['지점코드', '질병명'])['평균기온(°C)']
    .transform(lambda s: s.rolling(window=7, min_periods=1).mean())
)

# Lag/rolling 피처 때문에 생긴 초기 NaN은, 기후 중앙값 or 바로 직전값으로 채우기
lag_rolling_cols = [c for c in merged.columns if 'lag' in c or '7d_' in c]
for col in lag_rolling_cols:
    merged[col] = merged.groupby(['지점코드', '질병명'])[col].transform(
        lambda s: s.fillna(s.median())
    )

# =========================================
# 6. 주기성(계절성) Feature 생성
# =========================================

merged['month'] = merged['날짜'].dt.month
merged['dayofyear'] = merged['날짜'].dt.dayofyear
merged['weekofyear'] = merged['날짜'].dt.isocalendar().week.astype(int)

# sin/cos 인코딩 (1년 주기)
merged['sin_doy'] = np.sin(2 * np.pi * merged['dayofyear'] / 365.25)
merged['cos_doy'] = np.cos(2 * np.pi * merged['dayofyear'] / 365.25)

# (선택) 요일도 넣고 싶다면:
merged['weekday'] = merged['날짜'].dt.weekday  # 월=0 ~ 일=6

# =========================================
# 7. Feature Scaling (StandardScaler)
#    - 트리 계열(XGBoost 등)에는 필수는 아니지만,
#      신경망 계열(LSTM/TFT/MLP 등)에는 거의 필수.
# =========================================

# 스케일링 대상 Feature들 정의
feature_cols = (
    climate_numeric_cols
    + lag_rolling_cols
    + ['sin_doy', 'cos_doy', 'month', 'weekofyear']
)

# 존재하는 컬럼만 사용 (방어)
feature_cols = [c for c in feature_cols if c in merged.columns]

scaler = StandardScaler()
merged_scaled = merged.copy()
merged_scaled[feature_cols] = scaler.fit_transform(merged_scaled[feature_cols])

# 타겟 컬럼 (예: 일별발생건수)
target_col = '일별발생건수'

print("Final feature columns:", len(feature_cols))
print("Sample rows after scaling:")
print(merged_scaled[feature_cols + [target_col]].head())

# =========================================
# 8. 전처리 완료 데이터 저장 (모델링용)
# =========================================

OUTPUT_PATH = os.path.join(BASE_DIR, 'kdca_asos_daily_2023_2025_preprocessed.csv')
merged_scaled.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')

print(f"\n✅ 통합 전처리 데이터 저장 완료: {OUTPUT_PATH}")


✅ KDCA daily loaded: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/kdca_daily_2023_6regions.csv, shape=(28392, 14)
✅ KDCA daily loaded: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/kdca_daily_2024_6regions.csv, shape=(28392, 14)
✅ KDCA daily loaded: /content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터/kdca_daily_2025_6regions.csv, shape=(26754, 14)
KDCA daily combined: (83538, 14)
ASOS raw: (6432, 19)
Merged shape: (83538, 33)
After NA handling, any NaN in climate columns? False
Final feature columns: 40
Sample rows after scaling:
   평균기온(°C)  최저기온(°C)  최고기온(°C)  평균 이슬점온도(°C)  평균 상대습도(%)  최소 상대습도(%)  \
0 -1.525014 -1.366860 -1.456197     -1.659573   -1.069910   -1.347096   
1 -1.921647 -1.822511 -1.846867     -1.970657   -1.341339   -1.111563   
2 -1.997196 -1.992264 -1.865925     -2.004288   -1.137767   -1.191858   
3 -1.751662 -1.804642 -1.551483     -1.794096   -1.137767   -1.352449   
4 -1.695000 -1.742102 -1.437140     -1.667980   -0.812052   -1.38992

---

kdca_asos_merged_2023_2025_raw.csv를 기준으로
결측값 → Lag → 계절성 → 스케일링까지 한 번에 처리해서
최종 모델링용 CSV를 만들어주는 파이프라인

**전처리 + Feature Engineering 파트**

1. **결측값 처리**

  기후 수치 피처들을 지점코드 기준 중앙값 → 전체 중앙값 순서로 채움

  “관측 실패”를 0으로 보는 위험을 피함

2. **Lag Feature**

기온/습도/강수/일사에 대해

lag1, lag7, lag14

일강수량_7d_sum, 평균기온_7d_mean

→ “지난 1일/1주/2주 동안의 기후가 오늘 질병 발생에 미치는 영향”을 표현

3. **Seasonality Feature**

dayofyear, month, weekofyear, weekday

sin_doy, cos_doy로 연간 주기를 부드럽게 인코딩

→ “봄/여름/가을/겨울 패턴”을 모델이 이해할 수 있게 함

4. **Scaling**

StandardScaler로 기후·Lag·Seasonality를 정규화

→ 특히 LSTM/TFT/MLP 같은 신경망 모델에서 필수에 가까운 작업

In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# =========================================
# 0. 경로 및 파일 설정
# =========================================

BASE_DIR = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터'
INPUT_PATH = os.path.join(BASE_DIR, 'kdca_asos_merged_2023_2025_raw.csv')
OUTPUT_PATH = os.path.join(BASE_DIR, 'kdca_asos_2023_2025_preprocessed.csv')

# =========================================
# 1. 데이터 로드
# =========================================

df = pd.read_csv(INPUT_PATH)

# 날짜 컬럼 datetime 변환
df['날짜'] = pd.to_datetime(df['날짜'])

print("원본 데이터 shape:", df.shape)
print(df.head())

# =========================================
# 2. 결측값 처리 (기후 피처 위주)
# =========================================

# ASOS에서 가져온 기후 수치 컬럼들
climate_numeric_cols = [
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',
    '평균 이슬점온도(°C)',
    '평균 상대습도(%)',
    '최소 상대습도(%)',
    '평균 증기압(hPa)',
    '일강수량(mm)',
    '강수 계속시간(hr)',
    '1시간 최다강수량(mm)',
    '평균 풍속(m/s)',
    '최대 순간 풍속(m/s)',
    '합계 일조시간(hr)',
    '합계 일사량(MJ/m2)',
    '평균 지면온도(°C)',
    '안개 계속시간(hr)',
]

# 수치형으로 변환 (문자 → NaN)
for col in climate_numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 지점코드 기준으로 중앙값으로 채우기 (관측 실패값에 대한 보수적 처리)
for col in climate_numeric_cols:
    if col not in df.columns:
        continue
    df[col] = df.groupby('지점코드')[col].transform(
        lambda s: s.fillna(s.median())
    )

# 그래도 남아있는 NaN은 전체 중앙값으로 한 번 더 처리
for col in climate_numeric_cols:
    if col in df.columns and df[col].isna().any():
        df[col] = df[col].fillna(df[col].median())

print("\n기후 컬럼 결측 여부:")
print(df[climate_numeric_cols].isna().sum())

# =========================================
# 3. Lag Feature 생성 (지연 효과 반영)
# =========================================

# lag을 만들 주요 기후 피처 (너무 많지 않게 핵심만 선택)
lag_base_cols = [
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',
    '평균 상대습도(%)',
    '일강수량(mm)',
    '합계 일사량(MJ/m2)',
]

# 적용할 lag 일수 (1일, 7일, 14일 지연)
lags = [1, 7, 14]

# 정렬 (지점코드, 질병명, 날짜 기준 시계열 정렬)
df = df.sort_values(by=['지점코드', '질병명', '날짜'])

for col in lag_base_cols:
    if col not in df.columns:
        continue
    for lag in lags:
        df[f'{col}_lag{lag}'] = (
            df.groupby(['지점코드', '질병명'])[col]
              .shift(lag)
        )

# Rolling feature 예시 (7일 합/평균)
if '일강수량(mm)' in df.columns:
    df['일강수량_7d_sum'] = (
        df.groupby(['지점코드', '질병명'])['일강수량(mm)']
          .transform(lambda s: s.rolling(window=7, min_periods=1).sum())
    )

if '평균기온(°C)' in df.columns:
    df['평균기온_7d_mean'] = (
        df.groupby(['지점코드', '질병명'])['평균기온(°C)']
          .transform(lambda s: s.rolling(window=7, min_periods=1).mean())
    )

# lag/rolling에서 생긴 NaN 처리 (초기 구간)
lag_rolling_cols = [c for c in df.columns if 'lag' in c or '7d_' in c]

for col in lag_rolling_cols:
    df[col] = df.groupby(['지점코드', '질병명'])[col].transform(
        lambda s: s.fillna(s.median())
    )

print("\nLag / rolling feature 예시:")
print(df[lag_rolling_cols].head())

# =========================================
# 4. 계절성(Seasonality) Feature 생성
# =========================================

df['month'] = df['날짜'].dt.month
df['dayofyear'] = df['날짜'].dt.dayofyear
df['weekofyear'] = df['날짜'].dt.isocalendar().week.astype(int)
df['weekday'] = df['날짜'].dt.weekday  # 월=0 ~ 일=6

# 1년 주기 sin/cos 인코딩
df['sin_doy'] = np.sin(2 * np.pi * df['dayofyear'] / 365.25)
df['cos_doy'] = np.cos(2 * np.pi * df['dayofyear'] / 365.25)

print("\nSeasonality feature 예시:")
print(df[['날짜', 'dayofyear', 'sin_doy', 'cos_doy']].head())

# =========================================
# 5. Scaling (StandardScaler)
# =========================================

# 스케일링할 feature들 (기후 + lag/rolling + 계절성 일부)
feature_cols = (
    climate_numeric_cols
    + lag_rolling_cols
    + ['sin_doy', 'cos_doy', 'month', 'weekofyear']
)

# 실제 존재하는 컬럼만 사용
feature_cols = [c for c in feature_cols if c in df.columns]

scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[feature_cols] = scaler.fit_transform(df_scaled[feature_cols])

print("\n스케일링 대상 feature 수:", len(feature_cols))
print("스케일링 후 예시:")
print(df_scaled[feature_cols].head())

# =========================================
# 6. 최종 저장 (모델링용 데이터셋)
# =========================================

df_scaled.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')
print(f"\n✅ 전처리 + Lag + Seasonality + Scaling 완료, 저장 경로: {OUTPUT_PATH}")


원본 데이터 shape: (83538, 30)
   지점명  지점코드  질병명  질병총합    년도  주차         날짜        주시작일        주종료일  주별발생건수  \
0  동두천    98  뎅기열    25  2023   1 2023-01-01  2023-01-01  2023-01-07       0   
1  동두천    98  뎅기열    25  2023   1 2023-01-02  2023-01-01  2023-01-07       0   
2  동두천    98  뎅기열    25  2023   1 2023-01-03  2023-01-01  2023-01-07       0   
3  동두천    98  뎅기열    25  2023   1 2023-01-04  2023-01-01  2023-01-07       0   
4  동두천    98  뎅기열    25  2023   1 2023-01-05  2023-01-01  2023-01-07       0   

   ...  평균 증기압(hPa)  일강수량(mm)  강수 계속시간(hr)  1시간 최다강수량(mm)  평균 풍속(m/s)  \
0  ...          2.7       NaN          NaN            NaN         1.1   
1  ...          1.8       NaN          NaN            NaN         0.6   
2  ...          1.8       NaN          NaN            NaN         0.5   
3  ...          2.2       NaN          NaN            NaN         0.3   
4  ...          2.5       NaN          NaN            NaN         0.3   

   최대 순간 풍속(m/s)  합계 일조시간(hr)  합계 일사량(MJ/m2)  평균 지면온도(

In [13]:
# 스케일링 제외한 버전

import os
import pandas as pd
import numpy as np

# =========================================
# 0. 경로 및 파일 설정
# =========================================

BASE_DIR = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터'
INPUT_PATH = os.path.join(BASE_DIR, 'kdca_asos_merged_2023_2025_raw.csv')
OUTPUT_PATH = os.path.join(BASE_DIR, 'kdca_asos_2023_2025_preprocessed_noscale.csv')

# =========================================
# 1. 데이터 로드
# =========================================

df = pd.read_csv(INPUT_PATH)

# 날짜 컬럼 datetime 변환
df['날짜'] = pd.to_datetime(df['날짜'])

print("원본 데이터 shape:", df.shape)
print(df.head())

# =========================================
# 2. 결측값 처리 (기후 피처 위주)
# =========================================

# ASOS에서 가져온 기후 수치 컬럼들
climate_numeric_cols = [
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',
    '평균 이슬점온도(°C)',
    '평균 상대습도(%)',
    '최소 상대습도(%)',
    '평균 증기압(hPa)',
    '일강수량(mm)',
    '강수 계속시간(hr)',
    '1시간 최다강수량(mm)',
    '평균 풍속(m/s)',
    '최대 순간 풍속(m/s)',
    '합계 일조시간(hr)',
    '합계 일사량(MJ/m2)',
    '평균 지면온도(°C)',
    '안개 계속시간(hr)',
]

# 수치형으로 변환 (문자 → NaN)
for col in climate_numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 지점코드 기준으로 중앙값으로 채우기 (관측 실패값에 대한 보수적 처리)
for col in climate_numeric_cols:
    if col not in df.columns:
        continue
    df[col] = df.groupby('지점코드')[col].transform(
        lambda s: s.fillna(s.median())
    )

# 그래도 남아있는 NaN은 전체 중앙값으로 한 번 더 처리
for col in climate_numeric_cols:
    if col in df.columns and df[col].isna().any():
        df[col] = df[col].fillna(df[col].median())

print("\n기후 컬럼 결측 여부:")
print(df[climate_numeric_cols].isna().sum())

# =========================================
# 3. Lag Feature 생성 (지연 효과 반영)
# =========================================

# lag을 만들 주요 기후 피처 (너무 많지 않게 핵심만 선택)
lag_base_cols = [
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',
    '평균 상대습도(%)',
    '일강수량(mm)',
    '합계 일사량(MJ/m2)',
]

# 적용할 lag 일수 (1일, 7일, 14일 지연)
lags = [1, 7, 14]

# 정렬 (지점코드, 질병명, 날짜 기준 시계열 정렬)
df = df.sort_values(by=['지점코드', '질병명', '날짜'])

for col in lag_base_cols:
    if col not in df.columns:
        continue
    for lag in lags:
        df[f'{col}_lag{lag}'] = (
            df.groupby(['지점코드', '질병명'])[col]
              .shift(lag)
        )

# Rolling feature 예시 (7일 합/평균)
if '일강수량(mm)' in df.columns:
    df['일강수량_7d_sum'] = (
        df.groupby(['지점코드', '질병명'])['일강수량(mm)']
          .transform(lambda s: s.rolling(window=7, min_periods=1).sum())
    )

if '평균기온(°C)' in df.columns:
    df['평균기온_7d_mean'] = (
        df.groupby(['지점코드', '질병명'])['평균기온(°C)']
          .transform(lambda s: s.rolling(window=7, min_periods=1).mean())
    )

# lag/rolling에서 생긴 NaN 처리 (초기 구간)
lag_rolling_cols = [c for c in df.columns if 'lag' in c or '7d_' in c]

for col in lag_rolling_cols:
    df[col] = df.groupby(['지점코드', '질병명'])[col].transform(
        lambda s: s.fillna(s.median())
    )

print("\nLag / rolling feature 예시:")
print(df[lag_rolling_cols].head())

# =========================================
# 4. 계절성(Seasonality) Feature 생성
# =========================================

df['month'] = df['날짜'].dt.month
df['dayofyear'] = df['날짜'].dt.dayofyear
df['weekofyear'] = df['날짜'].dt.isocalendar().week.astype(int)
df['weekday'] = df['날짜'].dt.weekday  # 월=0 ~ 일=6

# 1년 주기 sin/cos 인코딩
df['sin_doy'] = np.sin(2 * np.pi * df['dayofyear'] / 365.25)
df['cos_doy'] = np.cos(2 * np.pi * df['dayofyear'] / 365.25)

print("\nSeasonality feature 예시:")
print(df[['날짜', 'dayofyear', 'sin_doy', 'cos_doy']].head())

# =========================================
# 5. 최종 저장 (스케일링 없이, Feature만 만든 상태)
# =========================================

df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')
print(f"\n✅ 전처리 + Lag + Seasonality까지 완료 (No Scaling), 저장 경로: {OUTPUT_PATH}")


원본 데이터 shape: (83538, 30)
   지점명  지점코드  질병명  질병총합    년도  주차         날짜        주시작일        주종료일  주별발생건수  \
0  동두천    98  뎅기열    25  2023   1 2023-01-01  2023-01-01  2023-01-07       0   
1  동두천    98  뎅기열    25  2023   1 2023-01-02  2023-01-01  2023-01-07       0   
2  동두천    98  뎅기열    25  2023   1 2023-01-03  2023-01-01  2023-01-07       0   
3  동두천    98  뎅기열    25  2023   1 2023-01-04  2023-01-01  2023-01-07       0   
4  동두천    98  뎅기열    25  2023   1 2023-01-05  2023-01-01  2023-01-07       0   

   ...  평균 증기압(hPa)  일강수량(mm)  강수 계속시간(hr)  1시간 최다강수량(mm)  평균 풍속(m/s)  \
0  ...          2.7       NaN          NaN            NaN         1.1   
1  ...          1.8       NaN          NaN            NaN         0.6   
2  ...          1.8       NaN          NaN            NaN         0.5   
3  ...          2.2       NaN          NaN            NaN         0.3   
4  ...          2.5       NaN          NaN            NaN         0.3   

   최대 순간 풍속(m/s)  합계 일조시간(hr)  합계 일사량(MJ/m2)  평균 지면온도(

---

📌 지금 상태 요약

단계	작업 내용	완료 여부
1. 질병데이터 연도별 수집	KDCA 파일 추출	✔
2. 지역 매핑	경기 세부 지역 → 6개 기준지역	✔
3. 주별 컬럼 통합	COLUMN1~53 → long형 변환	✔
4. 주차 날짜 정보 생성	주시작일/주종료일 포함	✔
5. 일별 데이터 확장	주데이터 → 일데이터	✔
6. 기후데이터 필드 선정	OBS_ASOS_DD 주요 컬럼만 선택	✔
7. KDCA × ASOS Merge	날짜 + 지점 기준 결합	✔
8. 결측치 처리	중앙값 기반	✔
9. Lag Feature 생성	1/7/14일 지연 + Rolling	✔
10. Seasonality Feature 생성	sin/cos 기반 주기성	✔
11. Scaling	StandardScaler	✔
12. 모델링용 CSV 저장	kdca_asos_2023_2025_preprocessed.csv	✔

---

주 단위 모델링용 데이터셋 생성 + 일별 기후 세밀 정보(평균/합/극값/변동성/강수일수 등) 모두 포함

In [ ]:
import os
import pandas as pd
import numpy as np

# =========================================
# 0. 경로 및 파일 설정
# =========================================

BASE_DIR = '/content/drive/MyDrive/weather-sales-forecast-ai/23-25데이터'

# ⚠️ 여기 KDCA 주간 데이터 파일명은 실제 네가 만든 걸로 교체해줘
KDCA_WEEKLY_PATH = os.path.join(BASE_DIR, 'kdca_weekly_long_2023_2025_6regions_all.csv')

# ASOS 일별 관측 데이터
ASOS_DAILY_PATH = os.path.join(BASE_DIR, 'OBS_ASOS_DD.csv')

# ASOS에서 사용할 기후 컬럼들
ASOS_SELECTED_COLS = [
    '지점',            # ASOS 지점코드
    '지점명',
    '일시',            # 날짜

    # 🌡 온도 관련
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',

    # 💧 습도/증기압
    '평균 이슬점온도(°C)',
    '평균 상대습도(%)',
    '최소 상대습도(%)',
    '평균 증기압(hPa)',

    # ☔ 강수 관련
    '일강수량(mm)',
    '강수 계속시간(hr)',
    '1시간 최다강수량(mm)',

    # 💨 풍속 관련
    '평균 풍속(m/s)',
    '최대 순간 풍속(m/s)',

    # ☀ 일사 관련
    '합계 일조시간(hr)',
    '합계 일사량(MJ/m2)',

    # 🌫 지면/안개
    '평균 지면온도(°C)',
    '안개 계속시간(hr)',
]

# =========================================
# 1. KDCA 주간 질병 데이터 로드
#    (지점명/지점코드/년도/주차/주시작일/주종료일/주별발생건수 포함)
# =========================================

kdca_weekly = pd.read_csv(KDCA_WEEKLY_PATH)

# 날짜 컬럼 datetime 변환
kdca_weekly['주시작일'] = pd.to_datetime(kdca_weekly['주시작일'])
kdca_weekly['주종료일'] = pd.to_datetime(kdca_weekly['주종료일'])

# 지점코드 타입 정리 (ASOS '지점'과 맞추기 위해 정수형)
kdca_weekly['지점코드'] = pd.to_numeric(kdca_weekly['지점코드'], errors='coerce').astype('Int64')

print("KDCA weekly shape:", kdca_weekly.shape)
print(kdca_weekly.head())

# =========================================
# 2. KDCA 기준 주 캘린더 만들기
#    - 지점코드 × 년도 × 주차 × (주시작일~주종료일) → 주간에 포함되는 날짜(일~토) 리스트 생성
# =========================================

week_calendar = (
    kdca_weekly[['지점코드', '년도', '주차', '주시작일', '주종료일']]
    .drop_duplicates()
    .reset_index(drop=True)
)

# 각 주에 대해 해당 주 기간의 날짜 리스트 생성
week_calendar['날짜리스트'] = week_calendar.apply(
    lambda row: pd.date_range(start=row['주시작일'], end=row['주종료일'], freq='D'),
    axis=1
)

# 1주(1행) → 7일(7행)로 확장
week_calendar_daily = week_calendar.explode('날짜리스트').reset_index(drop=True)
week_calendar_daily = week_calendar_daily.rename(columns={'날짜리스트': '날짜'})

print("week_calendar_daily shape:", week_calendar_daily.shape)
print(week_calendar_daily.head())

# =========================================
# 3. ASOS 일별 기후 데이터 로드 + 필요한 컬럼만 선택
# =========================================

asos = pd.read_csv(ASOS_DAILY_PATH, encoding='cp949')

asos = asos[ASOS_SELECTED_COLS].copy()

# 일시 → 날짜
asos['일시'] = pd.to_datetime(asos['일시'])
asos = asos.rename(columns={'일시': '날짜'})

# 지점코드 타입을 KDCA와 맞춤
asos['지점'] = pd.to_numeric(asos['지점'], errors='coerce').astype('Int64')

print("ASOS daily shape:", asos.shape)
print(asos.head())

# 기후 수치 컬럼들을 numeric으로 변환
climate_numeric_cols = [
    '평균기온(°C)',
    '최저기온(°C)',
    '최고기온(°C)',
    '평균 이슬점온도(°C)',
    '평균 상대습도(%)',
    '최소 상대습도(%)',
    '평균 증기압(hPa)',
    '일강수량(mm)',
    '강수 계속시간(hr)',
    '1시간 최다강수량(mm)',
    '평균 풍속(m/s)',
    '최대 순간 풍속(m/s)',
    '합계 일조시간(hr)',
    '합계 일사량(MJ/m2)',
    '평균 지면온도(°C)',
    '안개 계속시간(hr)',
]

for col in climate_numeric_cols:
    if col in asos.columns:
        asos[col] = pd.to_numeric(asos[col], errors='coerce')

# =========================================
# 4. ASOS 일별 데이터에 KDCA 주차 정보 매핑
#    - 지점코드 + 날짜 기준으로 week_calendar_daily와 merge
#    - 이렇게 해야 "KDCA가 정의한 주(일~토)"에 정확히 맞춰짐
# =========================================

asos_with_week = pd.merge(
    asos,
    week_calendar_daily,
    how='inner',
    left_on=['지점', '날짜'],
    right_on=['지점코드', '날짜'],
)

print("asos_with_week shape:", asos_with_week.shape)
print(asos_with_week.head())

# =========================================
# 5. 주간 집계 Feature 생성
#    5-1) 기본 집계: 평균 / 합 / 최대
#    5-2) 확장 Feature: 변동성(std), 강수일수, 폭우일수, 일교차 등
# =========================================

# 5-1) 기본 집계용 dict
agg_dict_base = {
    # 온도/습도/압력: 주간 평균
    '평균기온(°C)': 'mean',
    '최저기온(°C)': 'mean',
    '최고기온(°C)': 'mean',
    '평균 이슬점온도(°C)': 'mean',
    '평균 상대습도(%)': 'mean',
    '최소 상대습도(%)': 'mean',
    '평균 증기압(hPa)': 'mean',
    '평균 지면온도(°C)': 'mean',
    '평균 풍속(m/s)': 'mean',

    # 강수/일조/일사/안개: 주간 합계
    '일강수량(mm)': 'sum',
    '강수 계속시간(hr)': 'sum',
    '합계 일조시간(hr)': 'sum',
    '합계 일사량(MJ/m2)': 'sum',
    '안개 계속시간(hr)': 'sum',

    # 극값(Extreme event): 주간 최대값
    '1시간 최다강수량(mm)': 'max',
    '최대 순간 풍속(m/s)': 'max',

    # 지점명: 대표값
    '지점명': 'first',
}

agg_dict_base = {k: v for k, v in agg_dict_base.items() if k in asos_with_week.columns}

# 5-2) 확장 Feature를 위한 일별 파생 컬럼 생성
asos_with_week['일교차(°C)'] = asos_with_week['최고기온(°C)'] - asos_with_week['최저기온(°C)']
asos_with_week['강수유무'] = (asos_with_week['일강수량(mm)'] > 0).astype(int)
asos_with_week['폭우유무_20mm'] = (asos_with_week['일강수량(mm)'] >= 20).astype(int)
asos_with_week['폭우유무_50mm'] = (asos_with_week['일강수량(mm)'] >= 50).astype(int)

# groupby key
group_keys = ['지점코드', '년도', '주차']

group = asos_with_week.groupby(group_keys)

# 5-3) 기본 집계 (평균/합/최대)
asos_weekly_base = group.agg(agg_dict_base).reset_index()

# 5-4) 확장 집계: 변동성(std), 횟수(count)
extra_features = []

# 평균기온 표준편차
temp_std = group['평균기온(°C)'].std().reset_index(name='평균기온(°C)_std')
extra_features.append(temp_std)

# 평균 상대습도 표준편차
humid_std = group['평균 상대습도(%)'].std().reset_index(name='평균 상대습도(%)_std')
extra_features.append(humid_std)

# 일교차 평균
diurnal_mean = group['일교차(°C)'].mean().reset_index(name='일교차(°C)_mean')
extra_features.append(diurnal_mean)

# 강수일수 (일강수량 > 0인 일 수)
rain_days = group['강수유무'].sum().reset_index(name='강수일수')
extra_features.append(rain_days)

# 폭우일수 (>=20mm, >=50mm)
heavy_rain_20 = group['폭우유무_20mm'].sum().reset_index(name='폭우일수_20mm이상')
extra_features.append(heavy_rain_20)

heavy_rain_50 = group['폭우유무_50mm'].sum().reset_index(name='폭우일수_50mm이상')
extra_features.append(heavy_rain_50)

# 5-5) base + extra features merge
asos_weekly = asos_weekly_base.copy()
for ef in extra_features:
    asos_weekly = asos_weekly.merge(ef, on=group_keys, how='left')

print("asos_weekly (기후 주간 Feature) shape:", asos_weekly.shape)
print(asos_weekly.head())

# =========================================
# 6. KDCA 주간 질병 데이터 + ASOS 주간 기후 Feature 결합
#    - 타겟: KDCA 주별발생건수
#    - Feature: 위에서 만든 주간 기후 요약(평균/합/극값/변동성/강수일수)
# =========================================

weekly_merged = pd.merge(
    kdca_weekly,
    asos_weekly,
    how='left',
    left_on=['지점코드', '년도', '주차'],
    right_on=['지점코드', '년도', '주차'],
)

# KDCA 쪽 지점명을 기준으로 사용하고, ASOS 지점명은 필요시만 활용
if '지점명_y' in weekly_merged.columns:
    weekly_merged = weekly_merged.drop(columns=['지점명_y'])
if '지점명_x' in weekly_merged.columns:
    weekly_merged = weekly_merged.rename(columns={'지점명_x': '지점명'})

print("weekly_merged shape:", weekly_merged.shape)
print(weekly_merged.head())

# =========================================
# 7. 최종 저장 (주 단위 모델링용 데이터셋)
#    - 이 파일을 가지고:
#      타겟 = 주별발생건수
#      Feature = 주간 기후 평균/합/최대/변동성/강수일수/폭우일수 등
#      으로 회귀/분류 모델 학습 가능
# =========================================

OUTPUT_PATH = os.path.join(BASE_DIR, 'kdca_asos_weekly_2023_2025_6regions_with_features.csv')
weekly_merged.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')

print(f"\n✅ 주 단위 KDCA + ASOS 기후 Feature 데이터셋 생성 완료: {OUTPUT_PATH}")


KDCA weekly shape: (12168, 9)
   지점명  지점코드  질병명  질병총합    년도  주차       주시작일       주종료일  주별발생건수
0  동두천    98  뎅기열    25  2023   1 2023-01-01 2023-01-07     0.0
1  동두천    98  뎅기열    25  2023   2 2023-01-08 2023-01-14     0.0
2  동두천    98  뎅기열    25  2023   3 2023-01-15 2023-01-21     1.0
3  동두천    98  뎅기열    25  2023   4 2023-01-22 2023-01-28     0.0
4  동두천    98  뎅기열    25  2023   5 2023-01-29 2023-02-04     0.0
week_calendar_daily shape: (6552, 6)
   지점코드    년도  주차       주시작일       주종료일         날짜
0    98  2023   1 2023-01-01 2023-01-07 2023-01-01
1    98  2023   1 2023-01-01 2023-01-07 2023-01-02
2    98  2023   1 2023-01-01 2023-01-07 2023-01-03
3    98  2023   1 2023-01-01 2023-01-07 2023-01-04
4    98  2023   1 2023-01-01 2023-01-07 2023-01-05
ASOS daily shape: (6432, 19)
   지점  지점명         날짜  평균기온(°C)  최저기온(°C)  최고기온(°C)  평균 이슬점온도(°C)  평균 상대습도(%)  \
0  98  동두천 2023-01-01      -2.6      -6.5       3.8         -11.9        56.0   
1  98  동두천 2023-01-02      -6.8     -11.6      -0.3 